<a href="https://colab.research.google.com/github/netra-poonia/Incentive-Calculator-Colab-notebooks/blob/main/StateHead_Incentive_Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## `State Business Head Incentive Calculation`

In [11]:
import pandas as pd

class IncentiveCalculator:
    """
    Calculates incentives for State Business Heads based on performance metrics.

    Metrics considered:
    - Disbursement and AUM Growth (75% weight)
    - Productivity (Median Secured Disbursals per RM, 25% weight)
    - Portfolio Quality (Current % and Early Delinquency %, adjustment factor)
    """

    def __init__(self, disbursement_aum_growth_percentage, median_secured_disbursals_per_rm, current_percentage, early_delinquency_percentage):
        """
        Initializes the IncentiveCalculator with performance metrics for a State Business Head.

        Args:
            disbursement_aum_growth_percentage: The percentage growth in disbursement and AUM
                                                relative to the target (e.g., 110 for 110% achievement).
            median_secured_disbursals_per_rm: The median secured disbursals per Relationship Manager
                                              in a given period.
            current_percentage: The percentage of the portfolio that is in current status.
            early_delinquency_percentage: The percentage of the portfolio that is in early delinquency.
        """
        self.disbursement_aum_growth_percentage = disbursement_aum_growth_percentage
        self.median_secured_disbursals_per_rm = median_secured_disbursals_per_rm
        self.current_percentage = current_percentage
        self.early_delinquency_percentage = early_delinquency_percentage

    def calculate_disbursement_aum_incentive(self):
        """
        Calculates the incentive based on the percentage of target achievement for Disbursement and AUM Growth.

        Assumes sample base and bonus payout values for different achievement slabs.
        The slabs and payouts are hardcoded within the method for simplicity.

        Returns:
            A tuple containing the calculated incentive amount for disbursement and AUM growth
            and a dictionary with the base and bonus values from the matching slab.
        """
        disbursement_aum_slabs = {
            ( 0, 75)  : {'base': 0, 'bonus': 0},
            (75, 90)  : {'base': 110000, 'bonus': 1400},
            (90, 100): {'base': 115000, 'bonus': 2250},
            (100, 120): {'base': 125000, 'bonus': 2750},
            (120, 140): {'base': 140000, 'bonus': 3000},
            (140, float('inf')): {'base': 160000, 'bonus': 3200}
        }

        incentive = 0
        applied_payouts = {'base': 0, 'bonus': 0}
        # Iterate through the defined slabs to find the applicable one
        for (lower_bound, upper_bound), payouts in disbursement_aum_slabs.items():
            if lower_bound <= self.disbursement_aum_growth_percentage < upper_bound:
                incentive = payouts['base'] + payouts['bonus']
                applied_payouts = payouts
                break # Found the correct slab, exit the loop

        return (incentive, applied_payouts)

    def calculate_productivity_incentive(self):
        """
        Calculates the incentive based on the median secured disbursals per RM.

        Assumes sample payout values for different ranges of median secured disbursals.
        The ranges and payouts are hardcoded within the method.

        Returns:
            A tuple containing the calculated productivity incentive amount and the
            specific payout value from the matching bucket.
        """
        # Define ranges and corresponding payout values for Productivity
        productivity_buckets = {
            (0, 3): 0,
            (3, 5): 120000,
            (5, 6): 150000,
            (6, 9): 225000,
            (10, float('inf')): 300000
        }

        incentive = 0
        applied_payout = 0
        # Iterate through the defined buckets to find the applicable one
        for (lower_bound, upper_bound), payout in productivity_buckets.items():
            if lower_bound <= self.median_secured_disbursals_per_rm < upper_bound:
                incentive = payout
                applied_payout = payout
                break # Found the correct bucket, exit the loop

        return (incentive, applied_payout)

    def calculate_portfolio_quality_adjustment(self):
        """
        Calculates the adjustment factor based on the Current % and Early Delinquency %.

        Assumes sample adjustment factor values based on a predefined matrix.
        The matrix is hardcoded within the method.

        Returns:
            A tuple containing the calculated portfolio quality adjustment factor
            and the specific adjustment factor value that was applied.
        """
        # Sample portfolio quality adjustment matrix:
        # Keys are tuples representing (Current % lower bound, Current % upper bound)
        # Values are dictionaries where keys are tuples representing (Early Delinquency % lower bound, Early Delinquency % upper bound)
        # and values are the adjustment factors.
        portfolio_quality_matrix = {
            (0, 96): {
                (0, 1): -0.25,
                (1, 2): -0.30,
                (2, float('inf')): -0.4
            },
            (96, 98): {
                (0, 1): -0.15,
                (1, 2): -0.20,
                (2, float('inf')): -0.30
            },
            (98, 99): {
                (0, 1): 0,
                (1, 2): -0.15,
                (2, float('inf')): -0.25
            },
            (99, float('inf')): {
                (0, 1): 0.1,
                (1, 2): 0,
                (2, float('inf')): -0.05
            }
        }

        adjustment_factor = 1.0  # Default to no adjustment if no matching bucket is found (should not happen with comprehensive buckets)
        applied_adjustment_factor = 1.0

        # Iterate through the Current % buckets
        for (current_lower, current_upper), delinquency_buckets in portfolio_quality_matrix.items():
            if current_lower <= self.current_percentage < current_upper:
                # If Current % matches a bucket, iterate through the Early Delinquency % buckets
                for (delinquency_lower, delinquency_upper), factor in delinquency_buckets.items():
                    if delinquency_lower <= self.early_delinquency_percentage < delinquency_upper:
                        adjustment_factor = factor
                        applied_adjustment_factor = factor
                        break  # Found the delinquency bucket, break the inner loop
                break  # Found the current percentage bucket, break the outer loop

        return (adjustment_factor, applied_adjustment_factor)


    def calculate_final_incentive(self):
        """
        Calculates the final incentive by combining weighted incentives from Disbursement/AUM and Productivity
        and applying the portfolio quality adjustment factor, and provides a breakdown.

        Disbursement and AUM Growth has a 75% weight.
        Productivity has a 25% weight.
        The sum of weighted incentives is multiplied by the portfolio quality adjustment factor.

        Returns:
            A dictionary containing the final calculated incentive and a breakdown of
            the components: disbursement/AUM incentive, productivity incentive,
            portfolio quality adjustment factor, disbursement/AUM base payout,
            disbursement/AUM bonus payout, productivity payout, and the applied
            portfolio quality adjustment value.
        """
        # Calculate incentives and breakdown for each component
        disbursement_aum_incentive, disbursement_aum_payouts = self.calculate_disbursement_aum_incentive()
        productivity_incentive, productivity_payout = self.calculate_productivity_incentive()
        portfolio_quality_adjustment, portfolio_quality_adjustment_applied = self.calculate_portfolio_quality_adjustment()

        # Calculate the weighted sum of Disbursement/AUM and Productivity incentives
        weighted_incentive = (disbursement_aum_incentive * 0.75) + (productivity_incentive * 0.25)

        # Apply the portfolio quality adjustment factor to the weighted incentive
        final_incentive = weighted_incentive * portfolio_quality_adjustment

        # Return a dictionary with the final incentive and the breakdown
        return {
            'final_incentive': final_incentive,
            'disbursement_aum_incentive': disbursement_aum_incentive,
            'productivity_incentive': productivity_incentive,
            'portfolio_quality_adjustment_factor': portfolio_quality_adjustment,
            'disbursement_aum_base_payout': disbursement_aum_payouts.get('base', 0),
            'disbursement_aum_bonus_payout': disbursement_aum_payouts.get('bonus', 0),
            'productivity_payout': productivity_payout,
            'portfolio_quality_adjustment_applied': portfolio_quality_adjustment_applied
        }

def process_bulk_incentives(data):
    """
    Processes a bulk dataset of State Business Heads to calculate incentives and breakdown.

    Iterates through a list of performance data dictionaries, creates an
    IncentiveCalculator instance for each, and calculates the final incentive
    with a breakdown.

    Args:
        data: A list of dictionaries, where each dictionary contains performance metrics
              for a State Business Head with keys matching the IncentiveCalculator
              __init__ arguments ('Disbursement and AUM Growth %',
              'Median Secured Disbursals per RM', 'Current %', 'Early Delinquency %').

    Returns:
        A list of dictionaries, where each dictionary contains the calculated final
        incentive and the breakdown for a business head.
    """
    results = []
    # Process each business head's data in the list
    for business_head_data in data:
        # Create an instance of the IncentiveCalculator with the current business head's data
        calculator = IncentiveCalculator(
            business_head_data['Disbursement and AUM Growth %'],
            business_head_data['Median Secured Disbursals per RM'],
            business_head_data['Current %'],
            business_head_data['Early Delinquency %']
        )
        # Calculate the final incentive and breakdown for the current business head
        final_incentive_details = calculator.calculate_final_incentive()
        # Append the calculated incentive details to the results list
        results.append(final_incentive_details)
    return results


def process_single_incentive(disbursement_aum_growth_percentage, median_secured_disbursals_per_rm, current_percentage, early_delinquency_percentage):
    """
    Calculates the incentive and breakdown for a single State Business Head.

    Creates an IncentiveCalculator instance with the provided performance metrics
    and calculates the final incentive with a breakdown.

    Args:
        disbursement_aum_growth_percentage: Percentage growth in disbursement and AUM.
        median_secured_disbursals_per_rm: Median secured disbursals per Relationship Manager.
        current_percentage: Percentage of current portfolio.
        early_delinquency_percentage: Percentage of early delinquency portfolio.

    Returns:
        A dictionary containing the final calculated incentive and the breakdown.
    """
    # Create an instance of the IncentiveCalculator with the single business head's data
    calculator = IncentiveCalculator(
        disbursement_aum_growth_percentage,
        median_secured_disbursals_per_rm,
        current_percentage,
        early_delinquency_percentage
    )
    # Calculate the final incentive and breakdown
    final_incentive_details = calculator.calculate_final_incentive()
    return final_incentive_details

def read_performance_data(file_path):
    """
    Reads performance data from a CSV file into a pandas DataFrame and prepares it.

    Args:
        file_path: The path to the CSV file.

    Returns:
        A pandas DataFrame containing the performance data.
    """
    df = pd.read_csv(file_path)
    # Assuming column names in the CSV match the required keys.
    # If not, add renaming logic here, e.g.,
    # df.rename(columns={'Old Column Name': 'Disbursement and AUM Growth %'}, inplace=True)
    return df

def process_bulk_incentives(df):
    """
    Processes a bulk dataset of State Business Heads from a DataFrame to calculate
    incentives and breakdown.

    Iterates through the DataFrame rows, creates an IncentiveCalculator instance
    for each, and calculates the final incentive with a breakdown.

    Args:
        df: A pandas DataFrame containing performance metrics for State Business Heads
            with columns matching the IncentiveCalculator __init__ arguments
            ('Disbursement and AUM Growth %', 'Median Secured Disbursals per RM',
             'Current %', 'Early Delinquency %').

    Returns:
        A pandas DataFrame where each row contains the calculated final incentive
        and the breakdown for a business head.
    """
    results_list = []
    # Iterate through the DataFrame rows
    for index, row in df.iterrows():
        # Create an instance of the IncentiveCalculator with the current row's data
        calculator = IncentiveCalculator(
            row['Disbursement and AUM Growth %'],
            row['Median Secured Disbursals per RM'],
            row['Current %'],
            row['Early Delinquency %']
        )
        # Calculate the final incentive and breakdown for the current business head
        final_incentive_details = calculator.calculate_final_incentive()
        # Append the calculated incentive details to the results list
        results_list.append(final_incentive_details)

    # Convert the list of dictionaries to a pandas DataFrame
    results_df = pd.DataFrame(results_list)
    return results_df

# The process_single_incentive function remains unchanged as per the task breakdown.
# The read_performance_data function remains unchanged as per the task breakdown.

def write_incentive_results(results_df, output_file_path):
    """
    Writes the incentive results DataFrame to a CSV file.

    Args:
        results_df: A pandas DataFrame containing the incentive results.
        output_file_path: The path where the output CSV file will be saved.
    """
    results_df.to_csv(output_file_path, index=False)

## `Final Calculation`

In [12]:
import os

# 1. Create a sample CSV file
sample_data = {
    'Disbursement and AUM Growth %': [110, 95, 130, 105],
    'Median Secured Disbursals per RM': [1200000, 750000, 1600000, 900000],
    'Current %': [85, 92, 78, 88],
    'Early Delinquency %': [1.5, 3.0, 6.0, 2.5]
}
sample_df = pd.DataFrame(sample_data)
sample_csv_path = 'sample_performance_data.csv'
sample_df.to_csv(sample_csv_path, index=False)
print(f"Created sample CSV file: {sample_csv_path}")

# 2. Call the read_performance_data function
performance_df = read_performance_data(sample_csv_path)
print("\nRead performance data from CSV:")
display(performance_df)

# 3. Call the process_bulk_incentives function
incentive_results_df = process_bulk_incentives(performance_df)
print("\nProcessed bulk incentives:")
display(incentive_results_df)

# 4. Define an output file path
output_csv_path = 'incentive_results.csv'

# 5. Call the write_incentive_results function
write_incentive_results(incentive_results_df, output_csv_path)

# 6. Print a confirmation message
print(f"\nBulk processing complete. Results saved to {output_csv_path}")

Created sample CSV file: sample_performance_data.csv

Read performance data from CSV:


,Disbursement and AUM Growth %,Median Secured Disbursals per RM,Current %,Early Delinquency %
0,110,1200000,85,1.5
1,95,750000,92,3.0
2,130,1600000,78,6.0
3,105,900000,88,2.5



Processed bulk incentives:


,final_incentive,disbursement_aum_incentive,productivity_incentive,portfolio_quality_adjustment_factor,disbursement_aum_base_payout,disbursement_aum_bonus_payout,productivity_payout,portfolio_quality_adjustment_applied
0,-51243.75,127750,300000,-0.3,125000,2750,300000,-0.3
1,-65175.00,117250,300000,-0.4,115000,2250,300000,-0.4
2,-72900.00,143000,300000,-0.4,140000,3000,300000,-0.4
3,-68325.00,127750,300000,-0.4,125000,2750,300000,-0.4



Bulk processing complete. Results saved to incentive_results.csv


In [13]:
# 1. Define a dictionary containing sample performance metrics for a single business head.
single_data = {
    'Disbursement and AUM Growth %': 105,
    'Median Secured Disbursals per RM': 900000,
    'Current %': 88,
    'Early Delinquency %': 2.5
}

# 2. Create an instance of the IncentiveCalculator class using the values from the single_data dictionary.
calculator = IncentiveCalculator(
    single_data['Disbursement and AUM Growth %'],
    single_data['Median Secured Disbursals per RM'],
    single_data['Current %'],
    single_data['Early Delinquency %']
)

# 3. Call the calculate_final_incentive() method on the IncentiveCalculator instance to get the detailed breakdown.
single_incentive_details = calculator.calculate_final_incentive()

# 4. Print the detailed breakdown dictionary to the console.
print("\nDetailed Breakdown for Single Business Head:")
print(single_incentive_details)


Detailed Breakdown for Single Business Head:
{'final_incentive': -68325.0, 'disbursement_aum_incentive': 127750, 'productivity_incentive': 300000, 'portfolio_quality_adjustment_factor': -0.4, 'disbursement_aum_base_payout': 125000, 'disbursement_aum_bonus_payout': 2750, 'productivity_payout': 300000, 'portfolio_quality_adjustment_applied': -0.4}
